In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import copy

from collections import defaultdict 

from synthesizer import Synthesizer
from generator import Generator
from algorithm import Algorithm

# warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 80

### Эксперимент

In [2]:
# seeds = 5 * np.arange(2, 6) + 111

#### Фиксированные параметры генератора
- series_type = "default"  // обычная линейная регрессия

- dim=10 // размерность сигнала
- low=-10, high=10 // границы весов для получения отклика
- noise_var=1 // дисперсия нормального шума
- workers_num=3 // число различных генераторов
- lower_bound=100, upper_bound=400 // границы длин каждого из кусков, получаемых генератором
- alternating=True // нет двух подряд кусков одного типа
#### Фиксированные параметры алгоритма
- from_start=False // начинаем подсчет всех потерь (и составного эксперта) не с самого начала, а после того как прошли по одному разу учатки каждого из  генераторов

- length=1500 // длина изучаемого участка временного ряда

- a=-40, b=40 // параметры алгоритма, отклики отсеиваем тоже по ним

#### Гиперпараметры эксперимента
- windows = [5, 10, 20] // варьируем длину окна обучения ($\frac12x, 1x, 2x$ соответственно, где $x$  &mdash; dim  сигнала )
- weights_function // функция начальных весов, cравниваем $\dfrac1{x^\alpha}$ для разных $\alpha$ и дефолтную $\dfrac{1}{(x+1)\ln^2(x+1)}$
- alpha_function // функция коэффицента $\alpha$ в Mixing Update, cравниваем $\dfrac1{(1 + x)^\alpha}$ для разных $\alpha$

In [3]:
from hypers import weight_hypers
from hypers import alpha_hypers 

from main import experiments
from main import df

100%|████████████████████████████████████████████████████████████████████████| 2375/2375 [00:57<00:00, 41.47it/s]


In [4]:
df

,noise,weight_function,alpha_function,mean,random_1,random_2,random_3,random_4
1,0.0001,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),91302.42,77975.47,85211.78,95428.76,106593.65
0,0.0001,1 / (x^1.01),1 / (x + 1),66131.05,56147.31,61467.85,68771.76,78137.28
3,0.001,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),91235.75,77770.31,85211.54,95365.74,106595.4
2,0.001,1 / (x^1.01),1 / (x + 1),66079.44,56010.39,61467.88,68701.5,78137.99
5,0.1,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),91329.98,77917.68,85156.01,95531.64,106714.59
4,0.1,1 / (x^1.01),1 / (x + 1),66036.04,55956.41,61360.3,68786.6,78040.85
7,1,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),93840.70,79840.65,87111.0,98238.14,110173.03
6,1,1 / (x^1.01),1 / (x + 1),66154.89,56259.47,60342.21,69245.43,78772.46
9,10,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),125609.49,89454.43,125170.45,143412.01,144401.08
8,10,1 / (x^1.01),1 / (x + 1),-7977.27,33132.17,-245166.0,90545.61,89579.14


In [5]:
# df.to_csv("df-big_windows.csv", sep='\t')

In [6]:
df_w = df[df["alpha_function"] == "1 / (x + 1)"].iloc[:, :4]
df_w

,noise,weight_function,alpha_function,mean
1,0.0001,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),91302.42
0,0.0001,1 / (x^1.01),1 / (x + 1),66131.05
3,0.001,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),91235.75
2,0.001,1 / (x^1.01),1 / (x + 1),66079.44
5,0.1,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),91329.98
4,0.1,1 / (x^1.01),1 / (x + 1),66036.04
7,1,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),93840.70
6,1,1 / (x^1.01),1 / (x + 1),66154.89
9,10,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),125609.49
8,10,1 / (x^1.01),1 / (x + 1),-7977.27


In [7]:
df_a = df[df["weight_function"] == "1 / (x^1.01)"].iloc[:,:4]
df_a

,noise,weight_function,alpha_function,mean
0,0.0001,1 / (x^1.01),1 / (x + 1),66131.05
2,0.001,1 / (x^1.01),1 / (x + 1),66079.44
4,0.1,1 / (x^1.01),1 / (x + 1),66036.04
6,1,1 / (x^1.01),1 / (x + 1),66154.89
8,10,1 / (x^1.01),1 / (x + 1),-7977.27
10,100,1 / (x^1.01),1 / (x + 1),-4902989.02


In [8]:
df[["noise", "mean"]].groupby("noise").mean()

,mean
noise,
0.0001,78716.735
0.0010,78657.595
0.1000,78683.010
1.0000,79997.795
10.0000,58816.110
100.0000,-4226190.185
